In [2]:
import numpy as np
import pandas as pd
# from sklearn import 

def load_data(path,hemisphere):
    X = pd.read_csv(path + hemisphere + '_nmf_input_raw.csv')
    X = X.to_numpy()
    return(X)


In [12]:
X = load_data('C:\\Users\\Jonah Kember\\Documents\\MAGeT\\', 'right')
print(X.shape)
print(X)

X = X.reshape(3,701520)

X.shape

X

(2220, 948)
[[0.72191167 0.79436788 0.75909817 ... 0.76418841 0.65398388 0.730506  ]
 [0.72510828 0.78546084 0.72786679 ... 0.88426632 0.65963888 0.76356181]
 [0.73222308 0.78458321 0.70771907 ... 1.02003666 0.64172678 0.78128285]
 ...
 [0.68405477 0.74138437 0.72555454 ... 0.65529586 0.56045389 0.6212992 ]
 [0.70787432 0.71979637 0.70232637 ... 0.61354036 0.55243068 0.53544196]
 [0.74960837 0.75619531 0.71799002 ... 0.57412933 0.57871644 0.46208222]]


array([[0.72191167, 0.79436788, 0.75909817, ..., 0.73226102, 0.58294741,
        0.68874833],
       [0.75351727, 0.72680681, 0.76405496, ..., 0.72871505, 0.62250623,
        0.70724514],
       [0.85902044, 0.96294143, 0.93273672, ..., 0.57412933, 0.57871644,
        0.46208222]])